<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>TSP</strong></h1>
  <h3><strong>Ejercicio 5</strong></h3>
</div>

Implementar el modelo MTZ o el modelo de flujo en redes (o ambos) para el problema del TSP vistos en clase con la siguiente instancia del problema:

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjunto de índices**

Modelo del TSP con variables de flujo:  
Sean $x_{ij} = 1$, $i, j \in N : i \neq j$ las variables de decisión que indican que se visita la ciudad $j$ inmediatamente después de la ciudad $i$ y $x_{ij} = 0$ en otro caso. Asimismo, sean $y_{ij} \geq 0$, $i, j \in N : i \neq j$ las variables de decisión que denotan el flujo de la ciudad $i$ a la ciudad $j$. Se considera el parámetro $b_i$, $i \in N$, con $b_1 = n - 1$ y $b_i = -1$, $i = 2, \ldots, n$.

### **Parámetros**

In [ ]:
tij = [
    [0, 26, 82, 65, 100, 145, 100],
    [66, 0, 56, 39, 109, 156, 80],
    [43, 57, 0, 16, 53, 100, 128],
    [27, 41, 62, 0, 97, 144, 37],
    [109, 135, 161, 174, 0, 47, 41],
    [157, 161, 114, 130, 60, 0, 23],
    [30, 101, 123, 89, 75, 105, 0]
]

n = len(tij)

### **Variables de decisión**

$$ y_{ij} \geq 0 \quad \forall i, j \in N : i \neq j $$


In [ ]:
y = {}
for i in range(n):
    for j in range(n):
        if i != j:
            y[i, j] = solver.NumVar(0, solver.infinity(), f'y[{i},{j}]')

$$ x_{ij} \in \{0, 1\} \quad \forall i, j \in N : i \neq j $$

In [ ]:
x = {}
for i in range(n):
    for j in range(n):
        if i != j:
            x[i, j] = solver.BoolVar(f'x[{i},{j}]')

### **Función objetivo**

$$ \min \sum_{i \in N} \sum_{j \in N : i \neq j} t_{ij} x_{ij} $$

In [ ]:
camino = solver.Objective()

camino = sum(tij[i][j] * x[i, j] for i in range(n) for j in range(n) if i != j)

solver.Minimize(camino)

### **Restricciones**

$$ \sum_{j \in N : i \neq j} x_{ij} = 1 \quad \forall i \in N $$

In [ ]:
for i in range(n):
    solver.Add(solver.Sum(x[i, j] for j in range(n) if i != j) == 1)

$$ \sum_{i \in N : i \neq j} x_{ij} = 1 \quad \forall j \in N $$

In [ ]:
for j in range(n):
    solver.Add(solver.Sum(x[i, j] for i in range(n) if i != j) == 1)

$$ \sum_{j \in N : i \neq j} y_{ij} - \sum_{j \in N : i \neq j} y_{ji} = b_i \quad \forall i \in N $$

In [ ]:
b = [n - 1] + [-1] * (n - 1) # b = [n - 1, -1, -1, ..., -1]
for i in range(n):
    solver.Add(solver.Sum(y[i, j] for j in range(n) if i != j) - solver.Sum(y[j, i] for j in range(n) if i != j) == b[i])


$$ y_{ij} \leq (n - 1) x_{ij} \quad \forall i, j \in N : i \neq j $$


In [ ]:
for i in range(n):
    for j in range(n):
        if i != j:
            solver.Add(y[i, j] <= (n - 1) * x[i, j])

### **Resolver**

In [ ]:
solver.Solve()

0

In [ ]:
print('Objective value =', solver.Objective().Value())
for i in range(n):
    for j in range(n):
        if i != j and x[i, j].solution_value() == 1:
            print(f'City {i} -> City {j}')

Objective value = 280.0
City 0 -> City 1
City 1 -> City 3
City 2 -> City 4
City 3 -> City 2
City 4 -> City 5
City 5 -> City 6
City 6 -> City 0
